In [7]:
import pandas as pd
from pathlib import Path

RAW = Path("../data/raw")
PROC = Path("../data/processed")
PROC.mkdir(parents=True, exist_ok=True)

#1) 엑셀 읽기
xlsx_path = RAW / "search_index_raw.xlsx"   # ✅ 변수명 + 확장자 둘 다 올바름
xls = pd.ExcelFile(xlsx_path)
print("시트 목록:", xls.sheet_names)

#첫 시트에 존재하는 데이터 읽기
df = pd.read_excel(xlsx_path, sheet_name=xls.sheet_names[0])
print("원본 칼럼:", list(df.columns))

#2) 날짜(기간) 칼럼 이름 감지
date_col_candidates = ["기간","날짜","Date","date"]
date_col = None
for c in date_col_candidates:
    if c in df.columns:
        date_col = c
        break
if date_col is None:
    df = pd.read_excel(xlsx_path, sheet_name=xls.sheet_names[0], header=1)
    print("헤더를 한 줄 내린 뒤 칼럼:", list(df.columns))
    for c in date_col_candidates:
        if c in df.columns:
            date_col = c
            break


assert date_col is not None, "날짜/기간 칼럼을 찾지 못했어요. df.head()로 형태로 보여주세요"

#3) 브랜드 칼럼 후보(한글명 기준)
brand_cols = ["롬앤","클리오","에뛰드","이니스프리","닥터자르트"]
present_brands = [c for c in brand_cols if c in df.columns]
assert present_brands, "브랜드 칼럼을 찾지 못했어요. df.columns를 확인해 칼럼명을 알려주세요"

#4) 필요한 칼럼만 추출 & 칼럼 표준화
use = [date_col] + present_brands
df2 = df[use].copy()
df2.rename(columns={date_col:"date"}, inplace=True)
df2["date"] = pd.to_datetime(df2["date"])

#5) 롱포맷으로 변환(brand, value)
long = df2.melt(id_vars="date", value_vars=present_brands,
                 var_name="brand_ko", value_name="search_index")

#영문 키로도 하나 만들기 (분석 편의)
ko2en = {"롬앤":"romand","클리오":"clio","에뛰드":"etude","이니스프리":"innisfree","닥터자르트":"drjart"}
long["brand"] = long["brand_ko"].map(ko2en).fillna(longp["brand_ko"])

#6) 최근 3년만 필터
long = long.sort_values("data")

#7)저장
out_path = PROC / "search_index.csv"
long.to_csv(out_path, index=False, encoding="utf-8")
print(f"저장 완료 -> {out_path}")
long.head()

시트 목록: ['개요']
원본 칼럼: ['url', 'http://datalab.naver.com/keyword/trendResult.naver?hashKey=N_8e4ecd761f63d62d63bf1f7c3fb33ced', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']
헤더를 한 줄 내린 뒤 칼럼: ['주제', '통검', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7', 'Unnamed: 8', 'Unnamed: 9']


C:\Users\hwayo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
C:\Users\hwayo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


AssertionError: 날짜/기간 칼럼을 찾지 못했어요. df.head()로 형태로 보여주세요

In [8]:
import pandas as pd
from pathlib import Path

xlsx_path = Path("../data/raw/search_index_raw.xlsx")

xls = pd.ExcelFile(xlsx_path)
print("전체 시트 목록:", xls.sheet_names)

# 각 시트별로 맨 윗부분 3줄씩 출력해보기
for sheet in xls.sheet_names:
    print("\n📄 시트명:", sheet)
    try:
        df = pd.read_excel(xlsx_path, sheet_name=sheet, nrows=5)
        print(df.head())
    except Exception as e:
        print("읽기 실패:", e)


전체 시트 목록: ['개요']

📄 시트명: 개요
   url  \
0   주제   
1   범위   
2   기간   
3   성별   
4  연령대   

  http://datalab.naver.com/keyword/trendResult.naver?hashKey=N_8e4ecd761f63d62d63bf1f7c3fb33ced  
0                                                 통검                                             
1                                                 합계                                             
2                             주간 : 2022-01 ~ 2025-10                                             
3                                          전체(여성,남성)                                             
4                                                 전체                                             


C:\Users\hwayo\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
